In [1]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

IMG_SIZE = 512

def load_images_from_folder(folder_path, label, img_size):
    data = []
    labels = []
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = load_img(img_path, target_size=(img_size, img_size))
        img_array = img_to_array(img) / 255.0
        data.append(img_array)
        labels.append(label)
    return data, labels

# Load training data
train_is_path = "ice_cream/train/ice_cream"
train_not_path = "ice_cream/train/non_ice_cream"
train_data_is, labels_is = load_images_from_folder(train_is_path, 1, IMG_SIZE)
train_data_not, labels_not = load_images_from_folder(train_not_path, 0, IMG_SIZE)

X_train = np.array(train_data_is + train_data_not)
y_train = np.array(labels_is + labels_not)

# Load testing data
test_is_path = "ice_cream/test/ice_cream"
test_not_path = "ice_cream/test/non_ice_cream"
test_data_is, test_labels_is = load_images_from_folder(test_is_path, 1, IMG_SIZE)
test_data_not, test_labels_not = load_images_from_folder(test_not_path, 0, IMG_SIZE)

X_test = np.array(test_data_is + test_data_not)
y_test = np.array(test_labels_is + test_labels_not)


In [ ]:
from sklearn.utils import shuffle
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Data information
print("Data Shapes:")
print(f"x_train: {X_train.shape} | y_train: {y_train.shape}")
print(f"x_test: {X_test.shape} | y_test: {y_test.shape}\n")

# Shuffle data
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

# Enhanced CNN model with regularization
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),  # Added dropout for regularization
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),  # Increased dropout
    
    layers.Conv2D(128, (3, 3), activation='relu'),  # Added more filters
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),  # Increased units
    layers.Dropout(0.5),  # Higher dropout for dense layer
    layers.Dense(1, activation='sigmoid')
])

# Compile with lower learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

# Early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train model with validation
print("Training model...\n")
history = model.fit(
    X_train, y_train,
    epochs=30,  # Increased epochs
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

model.summary()

# Evaluate model
print("\nModel Evaluation:")
test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test)
test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print(f'\nTest accuracy: {test_acc:.4f}')
print(f'Test precision: {test_precision:.4f}')
print(f'Test recall: {test_recall:.4f}')
print(f'Test F1-score: {test_f1:.4f}')

# Generate predictions
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype("int32")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, 
                          target_names=['Not Ice Cream', 'Ice Cream']))

# Enhanced confusion matrix plot
plt.figure(figsize=(10, 8))
conf_mat = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Not Ice Cream', 'Ice Cream'],
            yticklabels=['Not Ice Cream', 'Ice Cream'],
            annot_kws={"size": 14})
plt.title('Confusion Matrix', fontsize=16)
plt.ylabel('True Label', fontsize=14)
plt.xlabel('Predicted Label', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

# Enhanced training history plots
plt.figure(figsize=(14, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation', linewidth=2)
plt.title('Model Accuracy', fontsize=14)
plt.ylabel('Accuracy', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation', linewidth=2)
plt.title('Model Loss', fontsize=14)
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Save model
model.save('ice_cream_classifier_improved.h5')
print("\nModel saved as 'ice_cream_classifier_improved.h5'")

Data Shapes:
x_train: (1800, 512, 512, 3) | y_train: (1800,)
x_test: (200, 512, 512, 3) | y_test: (200,)



c:\Users\hassa\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training model...

Epoch 1/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 260s 4s/step - accuracy: 0.5454 - loss: 1.3397 - precision: 0.5315 - recall: 0.4835 - val_accuracy: 0.7550 - val_loss: 0.5967 - val_precision: 0.7742 - val_recall: 0.7200
Epoch 2/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 200s 3s/step - accuracy: 0.7036 - loss: 0.5848 - precision: 0.7353 - recall: 0.6568 - val_accuracy: 0.7650 - val_loss: 0.5714 - val_precision: 0.7912 - val_recall: 0.7200
Epoch 3/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 200s 4s/step - accuracy: 0.7190 - loss: 0.5586 - precision: 0.7315 - recall: 0.7165 - val_accuracy: 0.7800 - val_loss: 0.5508 - val_precision: 0.7456 - val_recall: 0.8500
Epoch 4/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 195s 3s/step - accuracy: 0.7264 - loss: 0.5338 - precision: 0.7165 - recall: 0.7408 - val_accuracy: 0.7950 - val_loss: 0.5480 - val_precision: 0.7864 - val_recall: 0.8100
Epoch 5/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 193s 3s/step - accuracy: 0.7269 - loss: 0.5428 - precision: 0.7639 - recall: 0.6803 - val_accuracy: 0.7850 